In [1]:
import numpy as np
import pandas as pd
import jieba
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.externals import joblib

In [2]:
news = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')

In [3]:
news.dropna(inplace=True)

In [4]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76925 entries, 4 to 89610
Data columns (total 7 columns):
id         76925 non-null int64
author     76925 non-null object
source     76925 non-null object
content    76925 non-null object
feature    76925 non-null object
title      76925 non-null object
url        76925 non-null object
dtypes: int64(1), object(6)
memory usage: 4.7+ MB


In [5]:
news.replace({'source': r'^(新华社)'},{'source': 1}, regex=True, inplace=True)

In [6]:
news.loc[news['source'] != 1, 'source'] = 0

In [7]:
news.head(300)

,id,author,source,content,feature,title,url
4,89613,胡淑丽_MN7479,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...
5,89612,张怡,0,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",金融股一枝独秀 配置价值犹存,http://www.cs.com.cn/gppd/201706/t20170623_533...
7,89610,申玉彬 整理,0,沙漠雄鹰：震荡有利消化套牢筹码\r\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友早评：震荡有利消化套牢筹码,http://www.cs.com.cn/gppd/201706/t20170623_533...
8,89609,李杭_BJS4645,0,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""1600""...",武汉千余警察出动 抓获808名俊男靓女全是诈骗犯,http://news.163.com/17/0614/14/CMT9N8G80001899...
9,89608,吴瞬,0,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",纳入MSCI指数 A股长期配置价值提升,http://www.cs.com.cn/gppd/201706/t20170623_533...
13,89604,申玉彬 整理,0,曹先生：风格转换前的阵痛\r\n 今日早盘两地低开，之后一度震荡走高，领涨的仍然是上证...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",博友午评：风格转换前的阵痛,http://www.cs.com.cn/gppd/201706/t20170623_533...
15,89602,费天元,0,中国证券网讯（记者 费天元）沪深两市周五临近午盘出现快速跳水，沪指连续跌穿5日、10日及...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",地产股拉升难阻市场颓势 三大指数早盘集体跳水,http://www.cs.com.cn/gppd/201706/t20170623_533...
18,89599,张怡,0,中证网讯 （本报记者 张怡）今日，三板做市指数继续低开下行，盘中最低触及1057.91点...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",新三板午评：三板做市指数刷新阶段低点,http://www.cs.com.cn/gppd/201706/t20170623_533...
19,89598,周佳 整理,0,提示声明：\r\n 本文涉及的行业及个股分析来源于券商研究报告，仅为分析人士对该行业及...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",机构解析：下周热点板块及个股探秘（附股）,http://www.cs.com.cn/gppd/hyyj/201706/t2017062...
21,89596,周佳 整理,0,提示声明：\r\n 本文涉及的行业及个股分析来源于券商研究报告，仅为分析人士对该行业及...,"{""type"":""市场"",""site"":""中证网"",""commentNum"":""0"",""jo...",机构推荐：下周具备布局潜力金股,http://www.cs.com.cn/gppd/tzpj/201706/t2017062...


In [8]:
with open('./stop_words.utf8') as f: 
    stop_words =  [l.strip() for l in f]

In [9]:
def cut(string): return [word for word in list(jieba.cut(string)) if word not in stop_words]

In [10]:
all_contents = news['content']

In [11]:
all_contents = [' '.join(cut(s)) for s in all_contents]

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_8/f14fxxnn7w13cd0l5x9hl4bm0000gn/T/jieba.cache
Loading model cost 0.779 seconds.
Prefix dict has been built succesfully.


In [12]:
all_contents[0]

'原 标题 44 岁 女子 跑 深圳 约会 网友 拒 暴雨 中 裸身 奔走 \r\n 深圳 交警 微博称 昨日 清晨 交警 发现 女子 赤裸 上身 行走 南坪 快速 期间 轻生 年头 一辅警 发现 赶紧 披上 黄衣 一路 劝说 \r\n 事发 时 \r\n 到底 发生 \r\n 南 记者 带 一起 还原 现场 \r\n 南 记者 龙岗 大队 坂田 中队 见到 辅警 刘青 发现 女生 辅警 一位 外表 高大 帅气 说话 略带 腼腆 90 青年 \r\n 刘青 介绍 月 16 日 早上 时 36 分 正在 环城 南路 附近 值勤 接到 中队 一位 女子 裸身 进入 机动车 可能 危险 警情 骑着 铁骑 沿路 寻找 大概 花 十多分钟 南坪 大道 坂田 出口 龙岗 方向 逆行 辅道 发现 女子 \r\n 女子 身上 一丝不挂 地逆 车流 而行 时 走时 停 时 坐 时 躺 险象环生 刘青停 铁骑 一名 巡防 员 追 上去 发现 女子 情绪 低落 话 刘青 尝试 女子 交流 劝说 女子 离开 女子 愿意 接受 继续 缓慢 地往 南坪 快速路 主干道 走 \r\n 路边 已经 聚集 市民 围观 刺激 女子 情绪 刘青 巡防 员 一边 盯 女子 一边 驱赶 围观 群众 \r\n 现场 还原 \r\n 警方 提供 一份 视频 了解 16 日 早上 时 25 分 女子 出现 坂雪岗 大道 环城 南路 监控 视频 中 女子 穿着 白色 内裤 正 坂雪岗 大道 南坪 快速 方向 缓慢 走 \r\n 当时 上班 高峰期 十字路口 车流 已经 排起 长队 女子 出现 时 路上 市民 纷纷 驻足 观望 不少 车辆 放慢 速度 女子 市民 观望 停下 脚步 依然 缓慢 走 女子 行进 十字路口 中间 时 一辆 货车 挡住 镜头 女子 再次出现 镜头 时 发现 女子 已经 没 穿 内裤 全身 裸露 继续 南坪 快速 方向 走 记者 发现 视频 中 女子 周围 没有 尾随 上前 劝止 市民 \r\n 大清早 路上 看到 情况 \r\n 恐怕 没 办法 淡定 \r\n 面对 这一 情况 \r\n 刘青 表示 根本 不敢 心里 挺 别扭 感觉 尴尬 刘青 跟随 女子 南坪 快速路 主干道 时 女子 作出 意想不到 举动 突然 靠近 护栏 上面 跳下去 刘青 赶忙 冲上去 拉住 女子 手 控制 住 远离

In [ ]:
import pickle

In [ ]:
with open('news_content_words.pkl', 'wb') as f:
    pickle.dump(all_contents, f)

In [ ]:
with open('news_content_words.pkl', 'rb') as f:
    all_contents = pickle.load(f)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(max_features=50)

In [15]:
X = vectorizer.fit_transform(all_contents)

In [16]:
X.shape

(76925, 50)

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
y = pd.to_numeric(news['source']).values

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=0)

In [20]:
y_train.shape

(23077,)

In [21]:
knn_clf = KNeighborsClassifier()

In [22]:
param_grid = [{'n_neighbors': [i for i in range(1,3)]}]

In [23]:
grid_search = GridSearchCV(knn_clf,param_grid,n_jobs=-1,verbose=3, scoring='accuracy')

In [24]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/anaconda3/envs/Kstone/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   6 | elapsed:  1.6min remaining:  3.2min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  1.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'n_neighbors': [1, 2]}], pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring='accuracy',
       verbose=3)

In [25]:
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train,cv=3,scoring='accuracy')
print (scores.mean(), scores)

0.9652901550006279 [0.96438321 0.96788872 0.96359854]


In [26]:
knn = grid_search.best_estimator_

In [27]:
knn.predict(X_test)

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
scores = cross_val_score(grid_search.best_estimator_, X_test, y_test,cv=3,scoring='accuracy')
print(scores.mean(), scores)